[![Video Placeholder](./videolink.png)](https://youtu.be/9mWOVKnrMHc "Converting GPO's to Azure Policy with PowerShell")

# Converting Group Policy to Azure Policy Guest Configuration 

In this walk through, we create an Azure Policy Guest Configuration package from a Group Policy Object (GPO). Specifically, we work with the Windows Server 2019 Security Baseline. 

Scenario: Customer has a local group policy object, and they would like to define an equivalent Azure Policy. Using BaselineMAangement and GuestConfiguration packages, this translation is already possible. We walk through start to finish how customers can go from starting with a GPO to finishing with assigning the equivalent Azure Policy to an Azure VM.

# Part 1: Download GPO and Related Modules

In this first set of steps, we will install the Baseline Management module, related DSC resources, and Azure modules. Note that most users have some of these modules already downloaded, and installation can take time. Then, we use the MapGuidsToGpoNames script to view the downloaded GPO's.

In [ ]:
# Install the BaselineManagement module, Guest Configuration DSC resource module, and relevant Azure modules from PowerShell Gallery
Install-Module az.resources, az.policyinsights, az.storage, guestconfiguration, gpregistrypolicyparser, securitypolicydsc, auditpolicydsc, baselinemanagement -scope currentuser -Repository psgallery -AllowClobber -Force

In [ ]:
        Install-Module az.resources, az.policyinsights, az.storage, guestconfiguration, gpregistrypolicyparser, securitypolicydsc, auditpolicydsc, baselinemanagement -scope currentuser -Repository psgallery -AllowClobber -force
        Install-Module -Name 'securitypolicdsc' -RequiredVersion 2.10.0.0
        (Get-Content -Path $outputscriptlocation).Replace("Import-DSCResource -ModuleName 'SecurityPolicyDSC'", "Import-DSCResource -ModuleName 'SecurityPolicyDSC' -ModuleVersion '2.10.0.0'") | Set-Content -Path $outputscriptlocation


In [ ]:
# Download the 2019 Baseline files from https://docs.microsoft.com/windows/security/threat-protection/security-compliance-toolkit-10
New-Item -Path 'C:\git\policyfiles\downloads' -Type Directory
Invoke-WebRequest -Uri 'https://download.microsoft.com/download/8/5/C/85C25433-A1B0-4FFA-9429-7E023E7DA8D8/Windows%2010%20Version%201909%20and%20Windows%20Server%20Version%201909%20Security%20Baseline.zip' -Out C:\git\policyfiles\downloads\Server2019Baseline.zip

In [ ]:
# Unblock and expand the downloaded Server 2019 Baseline.
Unblock-File C:\git\policyfiles\downloads\Server2019Baseline.zip
Expand-Archive -Path C:\git\policyfiles\downloads\Server2019Baseline.zip -DestinationPath C:\git\policyfiles\downloads\

In [ ]:
# Show content details of downloaded GPOs
C:\git\policyfiles\downloads\Scripts\Tools\MapGuidsToGpoNames.ps1 -rootdir C:\git\policyfiles\downloads\GPOs\ -Verbose

# Part 2: Convert GPO to Azure Policy Guest Configuration

In the next set of steps, we use the Baseline Management Module to convert from Group Policy Object to Desired State Configuration. Then, we use the Guest Configuration module to convert the Desired State Configuration into an Azure Policy Guest Configuration Package.

In [ ]:
# Convert the Group Policy to Desired State Configuration using the Baseline Management Module.
ConvertFrom-GPO -Path 'C:\git\policyfiles\downloads\GPOs\{3657C7A2-3FF3-4C21-9439-8FDF549F1D68}\' -OutputPath 'C:\git\policyfiles\' -OutputConfigurationScript -Verbose

In [ ]:
# Rename, reformat, and run the converted scripts before creating a policy content package.
Rename-Item -Path C:\git\policyfiles\DSCFromGPO.ps1 -NewName C:\git\policyfiles\Server2019Baseline.ps1
(Get-Content -Path C:\git\policyfiles\Server2019Baseline.ps1).Replace('DSCFromGPO', 'Server2019Baseline') | Set-Content -Path C:\git\policyfiles\Server2019Baseline.ps1
(Get-Content -Path C:\git\policyfiles\Server2019Baseline.ps1).Replace('PSDesiredStateConfiguration', 'PSDscResources') | Set-Content -Path C:\git\policyfiles\Server2019Baseline.ps1
C:\git\policyfiles\Server2019Baseline.ps1

In [ ]:
# Create an Azure Policy Guest Configuration content package.
New-GuestConfigurationPackage -Name Server2019Baseline -Configuration c:\git\policyfiles\localhost.mof -Verbose

# Part 3: Publish Azure Policy Guest Configuration

In the final set of steps, we publish the Azure Policy Guest Configuration package to blob storage in a storage account. Then, we use the uri to publish the Azure Policy definition to our Azure Account. Prior to publishing, run Connect-AzAccount -Subscription xxxxx to connect with the appropriate account and subscription.

In [ ]:
function publish {
     param(
     [Parameter(Mandatory=$true)]
     $resourceGroup,
     [Parameter(Mandatory=$true)]
     $storageAccountName,
     [Parameter(Mandatory=$true)]
     $storageContainerName,
     [Parameter(Mandatory=$true)]
     $filePath,
     [Parameter(Mandatory=$true)]
     $blobName
     )

     # Get Storage Context
     $Context = Get-AzStorageAccount -ResourceGroupName $resourceGroup `
         -Name $storageAccountName | `
         ForEach-Object { $_.Context }

     # Upload file
     $Blob = Set-AzStorageBlobContent -Context $Context `
         -Container $storageContainerName `
         -File $filePath `
         -Blob $blobName `
         -Force

     # Get url with SAS token
     $StartTime = (Get-Date)
     $ExpiryTime = $StartTime.AddYears('3')  # THREE YEAR EXPIRATION
     $SAS = New-AzStorageBlobSASToken -Context $Context `
         -Container $storageContainerName `
         -Blob $blobName `
         -StartTime $StartTime `
         -ExpiryTime $ExpiryTime `
         -Permission rl `
         -FullUri

     # Output
     return $SAS
 }

In [ ]:
# Replace the $resourceGroup, $storageAccount, and $storageContainer values below.

$resourceGroup = 'gpodemo'
$storageAccount = 'gpodemostorage'
$storageContainer = 'gpodemocontainer'
$path = 'C:\Windows\System32\Server2019Baseline\Server2019Baseline.zip'
$blob = 'Server2019Baseline.zip'

In [ ]:
# Use the publish function with the assigned parameters to publish the Guest Configuration package to public blob storage.
$uri = publish -resourceGroup $resourceGroup -storageAccountName $storageAccount -storageContainerName $storageContainer -filePath $path -blobName $blob

In [ ]:
# Once a Guest Configuration custom policy package has been created and uploaded, create the Guest Configuration policy definition. Use the New-GuestConfigurationPolicy cmdlet to create the Guest Configuration.
New-GuestConfigurationPolicy -ContentUri $Uri -DisplayName 'Server 2019 Configuration Baseline' -Description 'Validation of using a completely custom baseline configuration for Windows VMs' -Path C:\git\policyfiles\policy -Platform Windows

In [ ]:
# Publish the policy definitions using the Publish-GuestConfigurationPolicy cmdlet. 
Publish-GuestConfigurationPolicy -Path C:\git\policyfiles\policy\ -Verbose